Generate training data for decoder training

In [1]:
import numpy as np
from tensorflow.keras.models import load_model
import imageio as io
from tqdm.notebook import tqdm

Load the final model with access to the latent space image

In [2]:
m = load_model(r"C:\BAGLS\Latent_space_image_model.h5", compile=False)

Set the data source for creating the latent space images

In [3]:
# Location of the training data and path to the saved model
TRAINING_PATH = "C:/BAGLS/training_512x256/"
DESTINATION_PATH = "C:/BAGLS/latent"

# All training images
N = 55750 

train_imgs = [TRAINING_PATH + str(i) + ".png" for i in range(N)]
train_segs = [TRAINING_PATH + str(i) + "_seg.png" for i in range(N)]

Compute the latent space images

In [4]:
tmp = []
latents = []

batch = 128

for i, fn in enumerate(tqdm(train_imgs)):
    im = np.asarray(io.imread(fn), dtype=np.float32) 
    
    if len(im.shape) == 3:
        print("Color!", end="\r")
        im = im @ (0.3, 0.59, 0.11)
        
    im = im / 127.5 - 1
    
    tmp.append(im)
    
    if i % batch == batch-1:
        tmp = np.asarray(tmp)
        print(f"Next batch! {i}", end="\r")
        _, l = m.predict(tmp[..., None])
        
        latents.extend(l.squeeze())
        tmp = []
        
tmp = np.asarray(tmp)
print(f"Next batch! {i}", end="\r")
_, l = m.predict(tmp[..., None])

latents.extend(l.squeeze())       
        
latents = np.asarray(latents)

  0%|          | 0/55750 [00:00<?, ?it/s]

In [9]:
train_latents = [DESTINATION_PATH +"/"+ str(i) + "_latent.png" for i in range(N)]

We will save the latent space images in `uint8` (see Figure 4)

In [10]:
max_latent = latents.max()

print(max_latent, latents.shape)

1.5473969 (55750, 32, 16)


In [11]:
for fn, im in tqdm(zip(train_latents, latents), total=latents.shape[0]):
    io.imwrite(fn, (im / max_latent * 255).astype(np.uint8))

  0%|          | 0/55750 [00:00<?, ?it/s]